In [1]:
print('Starting generic python imports')
from pathlib import Path
import numpy as np
import os
from time import time, sleep
import sys
from munch import Munch
import random

print('Doing FKMC imports')
from FKMC.montecarlo import *

## overall input parameters

#put the repeat axis at the top so that I can watch the data come in in realtime!
repeats = 100
J = 5
Ts = np.array([8, 1.8, 1])
U = 5
alpha = 1.25
Ns = [4,8,16,32,64,128,256]

MCMC_params = Munch(
        state = None, #this gets filled in later because it depends on N
        N_steps = int(5*1000),
        N_burn_in = int(1*1000), 
        thin = 100,
        logger = Eigenspectrum_IPR_all(bins = 1000, limit = 10),
        proposal = p_multi_site_uniform_reflect,
        accept_function = perturbation_accept,
        warnings = True,
    )


total_jobs = repeats * len(Ts)

#give information to the dispatch script
batch_params = Munch(total_jobs = total_jobs, Ts = Ts, Ns = Ns, repeats = repeats, structure_names = ('repeats', 'Ts'))

print('Getting environment variables')
slurm_job_id = int(os.getenv('SLURM_ARRAY_JOB_ID', -1))
slurm_task_id = int(os.getenv('SLURM_ARRAY_TASK_ID', random.random()*total_jobs))
max_slurm_task_id = int(os.getenv('SLURM_ARRAY_TASK_MAX', total_jobs - 1))

##sleep if necessary
t0 = time()
debug = (slurm_job_id == -1)
sleeptime = np.random.random() * (30)
print(f'Waiting for {sleeptime:.0f} seconds to randomise the finish time')
if not debug: sleep(sleeptime)

##using the value of slurm_task_id, decide what parameters this job will have
repeat_i, T_j = divmod(slurm_task_id, len(Ts))
T = Ts[T_j]

logs = np.empty(shape = len(Ns), dtype = object)

if debug:
    MCMC_params.N_steps = 100
    MCMC_params.N_burn_in = 0
    MCMC_params.thin = 1

parameters = Munch(N_sites = 'variable', t = 1, alpha = alpha, mu = 0, beta = 1/T, J = J, U = U, normalise = True)
print(f'''
params = {parameters},
mc_params = {MCMC_params}
''')
    
for i, N in enumerate(Ns):
    print(f'starting N = {N}')
    parameters.N_sites = N
    
    t1 = time()
    logs[i] = FK_mcmc(**MCMC_params, parameters = parameters)
    logs[i].time = time() - t1
    


filepath = Path(f'./{slurm_task_id}.npz') if not debug else Path(f'./test_save.npz')

print(f'Saving in {filepath}')
np.savez_compressed(filepath, 
    **batch_params, parameters = parameters, MCMC_params = MCMC_params, logs = logs, allow_pickle = True,
    desc = ''
    )
print(f'done in {time() - t0:.0f} seconds')

Starting generic python imports
Doing FKMC imports
Getting environment variables
Waiting for 19 seconds to randomise the finish time

params = Munch({'N_sites': 'variable', 't': 1, 'alpha': 1.25, 'mu': 0, 'beta': 0.125, 'J': 5, 'U': 5, 'normalise': True}),
mc_params = Munch({'state': None, 'N_steps': 100, 'N_burn_in': 0, 'thin': 1, 'logger': <FKMC.montecarlo.Eigenspectrum_IPR_all object at 0x7f06fc9fbc50>, 'proposal': <function p_multi_site_uniform_reflect at 0x7f06e03cd2f0>, 'accept_function': <function perturbation_accept at 0x7f06e03cd620>, 'warnings': True})

starting N = 4
0% through after 0.00m
10% through after 0.00m
20% through after 0.00m
30% through after 0.00m
40% through after 0.00m
50% through after 0.00m
60% through after 0.00m
70% through after 0.00m
80% through after 0.00m
90% through after 0.00m
Warning, p_acc = 0.675, Munch({'N_sites': 4, 't': 1, 'alpha': 1.25, 'mu': 0, 'beta': 0.125, 'J': 5, 'U': 5, 'normalise': True, 'J_matrix': 'suppressed for brevity'})
starting N